In [57]:
import numpy as np
import pandas as pd
import torch 
from torchvision import transforms 
from sklearn.model_selection import train_test_split
from PIL import Image
import glob
data = []
for file_name in glob.glob("/kaggle/input/gtsrb-german-traffic-sign-recognition-benchmark/Train/*/*.png"):
    im_class = int(file_name.split('/')[-2])
    data.append((im_class,file_name))
    

In [65]:
train =[]
test = []
for x,y in data:
    train.append(y)
    test.append(x)
   

In [66]:
len(test)

39209

In [67]:
x_train,x_valid ,y_train ,y_valid = train_test_split(train,test ,test_size= .2)

In [64]:
x_train[0]

9

In [90]:
class Dataset(torch.utils.data.Dataset):
    def __init__(self,image,label,transform =None):
        self.image =image
        self.label =label
        self.transform = transform
    def __len__(self):
        return len(self.label)
    
    def __getitem__ (self,index):
        img_name = self.image[index]
        x = Image.open(img_name)
        y= self.label[index]
        if self.transform != None :
            x = self.transform(x)
        return x,y    
        
        

In [91]:
trainset = Dataset(x_train,y_train)


In [95]:
trainset

In [93]:
h= 1000
w= 1000
for i in range(len(trainset)):
    x,y =trainset[i]
    h ,w =  min(np.shape(x)[0] ,h) ,min(np.shape(x)[1] ,w)
print(h,w)    

25 25


In [102]:
transform = transforms.Compose(
[transforms.Resize( (25,25)),
transforms.ToTensor(),
transforms.Normalize((.5,.5,.5),(.5,.5,.5))])

In [119]:
trainset = Dataset(x_train,y_train,transform)
validset = Dataset(x_valid,y_valid,transform)
trainloader = torch.utils.data.DataLoader(trainset,batch_size = 64 ,shuffle = True,num_workers = 2)
validloader = torch.utils.data.DataLoader(validset,batch_size = 64 ,num_workers = 2)

In [ ]:
len(trainloader)

In [105]:
from torch.nn import Flatten , Conv2d ,MaxPool2d,ReLU ,Linear



In [109]:
model = torch.nn.modules.Sequential(Conv2d(3,16,3,padding =  1),ReLU()
                                    ,Conv2d(16,32,3,padding =  1),ReLU(),
                                    Conv2d(32,64,3,padding =  1),ReLU(),Flatten(),
                                   Linear(64*25*25,10000),ReLU(),
                                   Linear(10000,1000),ReLU(),
                                   Linear(1000,43)) 
model.cuda()
loss_function = torch.nn.CrossEntropyLoss().cuda()
optimizer = torch.optim.SGD(model.parameters(),lr =1e-3 , momentum= .9)


In [120]:
dataloader = {"train":trainloader,"valid":validloader}


for epoch in range(30):
  print(f'start of epoch:{epoch}')
  for phase in ["train","valid"]:
        if phase =="train":
            model.train()
            print(phase)
        else:
            model.eval()
            print(phase)
        total_loss = 0
        total_correct= 0
        total_example =0     
            
        for iter,(x,y) in enumerate(dataloader[phase]):
            x= x.cuda()
            y= y.cuda()
            score= model(x)
            pred = torch.argmax(score ,axis = 1)
            correct = torch.sum(pred==y)
            loss = loss_function(score,y)
            if phase == "train" :
                loss.backward()     
                with torch.no_grad():
                    optimizer.step()
                    optimizer.zero_grad()  

            total_loss += loss.item()
            total_correct += correct.item()
            total_example += len(y)
            if iter % 100 == 0:
                print(total_loss/total_example,total_correct/total_example)
            

start of epoch:0
train
0.02422410435974598 0.546875
0.025960985904754097 0.519589552238806
0.022294382611173495 0.5783587905236908
valid
0.013289687223732471 0.734375
start of epoch:1
train
0.01080390252172947 0.734375
0.011847262685442 0.7646144278606966
0.01051466290878499 0.7927057356608479
valid
0.012827587313950062 0.75
start of epoch:2
train
0.009756703861057758 0.828125
0.006578602946238287 0.8696361940298507
0.006335615568870952 0.8787016832917706
valid
0.006084352266043425 0.859375
start of epoch:3
train
0.005229836329817772 0.921875
0.004711858808313177 0.9091262437810945
0.004356440428140436 0.9177057356608479
valid
0.005019561853259802 0.90625
start of epoch:4
train
0.002583162160590291 0.96875
0.0031000913117331134 0.9462064676616916
0.0029716611559098666 0.9490336658354115
valid
0.002394055249169469 0.96875
start of epoch:5
train
0.0014459129888564348 0.984375
0.0024796101688615868 0.9597325870646766
0.0024391071758572097 0.9581514962593516
valid
0.0031525795347988605 0.9

KeyboardInterrupt: 

In [121]:
torch.save(model.state_dict(),"cnn.pt")

In [ ]:
output = open("answer.csv",'w+')
output.write('Filename,ClassID\n')
with torch.no_grad()
    for itr ,(x,y)in enumerate(testloader):
        x= x.cuda()
        score =model(x)
        pred = torch.argmax(score,axis =1)
        for id , pred in zip(y,pred):
            output.write(f'{id},{pred}\n')
output.close        